<a href="https://colab.research.google.com/github/jwengr/dacon/blob/main/%EC%86%8C%EC%84%A4%20%EC%9E%91%EA%B0%80%20%EB%B6%84%EB%A5%98%20AI%20%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C/EDA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!unzip drive/My\ Drive/dacon/lg/dataset/data.zip -d drive/My\ Drive/dacon/lg/dataset

In [ ]:
!pip install pandas_profiling --upgrade

In [ ]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#pd.options.mode.chained_assignment = None

In [ ]:
defaultpath = 'drive/My Drive/dacon/lg'

그냥 csv를 읽을시 데이터 손실이 나기때문에(타입문제) str타입으로 읽어준다.

In [ ]:
err_df= pd.read_csv(defaultpath+'/dataset/train_err_data.csv',encoding='utf-8',dtype=np.str)
quality_df= pd.read_csv(defaultpath+'/dataset/train_quality_data.csv',encoding='utf-8',dtype=np.str)
problem_df= pd.read_csv(defaultpath+'/dataset/train_problem_data.csv',encoding='utf-8',dtype=np.str)

test_err_df= pd.read_csv(defaultpath+'/dataset/test_err_data.csv',encoding='utf-8',dtype=np.str)
test_quality_df= pd.read_csv(defaultpath+'/dataset/test_quality_data.csv',encoding='utf-8',dtype=np.str)
sample_submission= pd.read_csv(defaultpath+'/dataset/sample_submission.csv',encoding='utf-8',dtype=np.str)

pandas profiling이 메모리문제로 안됨-> 일단 데이터를 직접 확인하자

In [ ]:
problem_df.head()

,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000


In [ ]:
problem_df.describe()

,user_id,time
count,5429,5429
unique,5000,488
top,24407,20201130120000
freq,5,42


In [ ]:
problem_df.isna().sum()

user_id    0
time       0
dtype: int64

In [ ]:
sample_submission.head()

,user_id,problem
0,30000,0
1,30001,0
2,30002,0
3,30003,0
4,30004,0


In [ ]:
sample_submission.describe()

,user_id,problem
count,14999,14999
unique,14999,1
top,36567,0
freq,1,14999


In [ ]:
err_df.describe()

,user_id,time,model_nm,fwver,errtype,errcode
count,16554663,16554663,16554663,16554663,16554663,16554662
unique,15000,2566163,9,37,41,2805
top,24934,20201108023257,model_1,04.16.3553,23,1
freq,222186,631,5384491,5237816,2276515,8906967


In [ ]:
err_df.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [ ]:
err_df.isna().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     1
dtype: int64

In [ ]:
quality_df.describe()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
count,828624,828624,788544,684192,828624,788511,828624,828624,828604,828624,828624,828624,828624,828624,828624,828624
unique,4319,8281,27,753,31,798,1,1,4744,549,884,42,523,4200,12,14
top,20201110143000,15357,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0
freq,948,4224,163236,542790,670270,632469,828624,828624,428096,662217,757788,787812,796284,99828,672229,801156


In [ ]:
quality_df.head()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [ ]:
quality_df.isna().sum()

time               0
user_id            0
fwver          40080
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

quality data의 nan 값이 상당히 많기 때문에 이를 잘 처리해야한다.

problem data : 5000명의 유저가 에러를 신고한 시간이다. 
5000명의 유저가 신고했지만 기간은 480에 불과하다.
즉 같은 시간대에 여러 유저들이 불편함을 느끼고 신고하였다.
sample submission : 14999 유저의 에러신고여부를 예측해야한다
err data : 15000명의 유저가 에러를 겪은 시간대와 그 에러종류, 모델, 버전정보가 있다. 한명의 유저가 여러시간대에 여러 에러를 겪는다.
quality data : 4319의 시간기간에 시스템 퀄리티의 정보와 사용한 유저가 있다.

problem data 분석

In [ ]:
problem_df['year'] = problem_df['time'].apply(lambda x: x[:4])
problem_df['month'] = problem_df['time'].apply(lambda x:  x[4:6])
problem_df['day'] = problem_df['time'].apply(lambda x: x[6:8])
problem_df['hour'] = problem_df['time'].apply(lambda x: x[8:10])
problem_df['mmss'] = problem_df['time'].apply(lambda x: x[10:]) 

In [ ]:
problem_df.describe()

,user_id,time,year,month,day,hour,mmss
count,5429,5429,5429,5429,5429,5429,5429
unique,5000,488,1,1,30,24,1
top,24407,20201130120000,2020,11,30,12,0000
freq,5,42,5429,5429,287,627,5429


problem data는 2020년 11월 한달동안의 에러를 겪은 유저이다.
즉 11월 30일에 에러를 신고한 유저를 예측해야 한다.
mmss는 기록되지 않는다.  유저의 id구간은 10001~24998이다

err data 분석

In [ ]:
err_df['year'] = err_df['time'].apply(lambda x: x[:4])
err_df['month'] = err_df['time'].apply(lambda x:  x[4:6])
err_df['day'] = err_df['time'].apply(lambda x: x[6:8])
err_df['hour'] = err_df['time'].apply(lambda x: x[8:10])
err_df['mmss'] = err_df['time'].apply(lambda x: x[10:]) 

In [ ]:
err_df.describe()

,user_id,time,model_nm,fwver,errtype,errcode,year,month,day,hour,mmss
count,16554663,16554663,16554663,16554663,16554663,16554662,16554663,16554663,16554663,16554663,16554663
unique,15000,2566163,9,37,41,2805,1,3,31,24,3600
top,24934,20201108023257,model_1,04.16.3553,23,1,2020,11,21,19,0008
freq,222186,631,5384491,5237816,2276515,8906967,16554663,16554577,649909,899350,5377


err data 도 2020년인건 동일하지만 11월을 포함하여 3달간의 데이터입니다.

In [ ]:
err_df['time'].astype(int).describe()

count    1.655466e+07
mean     2.020112e+13
std      8.664375e+06
min      2.020103e+13
25%      2.020111e+13
50%      2.020112e+13
75%      2.020112e+13
max      2.020120e+13
Name: time, dtype: float64

err data는 10월부터 12월까지의 데이터 입니다.
11월 30일의 결과를 예측해야 하고 에러신고데이터도 11월 30일밖에 존재하지 않습니다.
다른 날짜들의 데이터가 도움이 될까요?
일단 11월30일의 데이터만 살펴보겠습니다.

In [ ]:
err_df[err_df['time'].str.contains('20201130.{6}')].describe()

,user_id,time,model_nm,fwver,errtype,errcode,year,month,day,hour,mmss
count,512635,512635,512635,512635,512635,512635,512635,512635,512635,512635,512635
unique,13884,85404,9,20,41,196,1,1,1,24,3600
top,16089,20201130093608,model_1,04.16.3553,31,1,2020,11,30,20,2501
freq,8393,45,143445,143289,68093,285740,512635,512635,512635,27856,219


err_df에서
11월 30일의 총 유저는 512535 이고 중복을 제거하면
13884명의 유저가 에러를 겪었다. 유저의 범위는 10000~24999이다. 

조사대상은 10000~24999 총 15000명의 유저입니다.
그 중 11월 30일에 사용한 유저는 13884명입니다.
11월 30일 에러를 신고한 유저는 5000명입니다. 딱 떨어지는게 이상합니다.

각 유저들의 에러 경험 수를 컬럼으로 만들고 완전중복인 행을 제거합니다.

In [ ]:
err_ppdf = err_df[err_df['time'].str.contains('20201130.{6}')].drop_duplicates()

In [ ]:
err_ppdf = err_ppdf.drop(['year','month','day','hour','time','mmss','hour'],axis=1)

In [ ]:
err_ppdf = err_ppdf.groupby(err_ppdf.columns.tolist()).size().reset_index().rename(columns={0:'duplicated'})

In [ ]:
oh = pd.get_dummies

In [ ]:
err_ppdf = pd.concat([err_ppdf['user_id'],
                      pd.concat([oh(err_ppdf['model_nm']),oh(err_ppdf['fwver']),oh(err_ppdf['errtype']),oh(err_ppdf['errcode'])
                      ],
                      axis=1
                      ).multiply(err_ppdf['duplicated'],axis='index')],axis=1)

In [ ]:
err_ppdf = err_ppdf.groupby('user_id').sum().reset_index()

quality data 분석

In [ ]:
quality_df['year'] = quality_df['time'].apply(lambda x: x[:4])
quality_df['month'] = quality_df['time'].apply(lambda x:  x[4:6])
quality_df['day'] = quality_df['time'].apply(lambda x: x[6:8])
quality_df['hour'] = quality_df['time'].apply(lambda x: x[8:10])
quality_df['mmss'] = quality_df['time'].apply(lambda x: x[10:]) 

In [ ]:
quality_df.describe()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,year,month,day,hour,mmss
count,828624,828624,788544,684192,828624,788511,828624,828624,828604,828624,828624,828624,828624,828624,828624,828624,828624,828624,828624,828624,828624
unique,4319,8281,27,753,31,798,1,1,4744,549,884,42,523,4200,12,14,1,2,31,24,6
top,20201110143000,15357,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0,2020,11,08,22,2000
freq,948,4224,163236,542790,670270,632469,828624,828624,428096,662217,757788,787812,796284,99828,672229,801156,828624,828408,33996,59352,151224


In [ ]:
quality_df['time'].astype(int).describe().astype(int)

count            828624
mean     20201115421121
std             8863638
min      20201031235000
25%      20201108102750
50%      20201115182500
75%      20201123131000
max      20201130234000
Name: time, dtype: int64

quality data는 2020년 10월 11일부터 2020년 11월 30일 23시까지의 로그를 가지고 있다.

In [ ]:
quality_ppdf = quality_df[quality_df['time'].str.contains('20201130.{6}')].drop(['time','year','month','day','mmss'],axis=1)

In [ ]:
quality_ppdf.isna().sum()

user_id          0
fwver         1308
quality_0     4788
quality_1        0
quality_2     1308
quality_3        0
quality_4        0
quality_5        0
quality_6        0
quality_7        0
quality_8        0
quality_9        0
quality_10       0
quality_11       0
quality_12       0
hour             0
dtype: int64

In [ ]:
for i in range(13):
    print('quality_'+str(i),set(quality_ppdf[quality_ppdf.notna()].drop(['user_id','fwver'],axis=1).iloc[:,i].values))

quality_0 {nan, '7.0', '11.0', '10.0', '3.0', '9.0', '24.0', '17.0', '1.0', '29.0', '14.0', '5.0', '4.0', '25.0', '19.0', '37.0', '20.0', '12.0', '-1.0', '57.0', '18.0', '0.0', '2.0', '21.0', '64928.0', '6.0', '121.0', '13.0', '8.0', '16.0'}
quality_1 {'5', '1', '18', '-1', '2', '0', '7', '3', '9', '21', '6', '4', '13', '8'}
quality_2 {nan, '7.0', '11.0', '10.0', '3.0', '9.0', '24.0', '17.0', '1.0', '29.0', '14.0', '5.0', '4.0', '30.0', '23.0', '25.0', '19.0', '37.0', '20.0', '12.0', '-1.0', '57.0', '18.0', '0.0', '2.0', '21.0', '64928.0', '6.0', '22.0', '121.0', '13.0', '8.0', '16.0'}
quality_3 {'0'}
quality_4 {'0'}
quality_5 {'1,858', '1,143', '1,463', '174', '155', '104', '524', '373', '469', '219', '8,895', '8', '434', '124', '462', '441', '1,222', '44', '1,580', '1,341', '4,931', '69', '23', '1,168', '396', '490', '6,945', '1,975', '1,830', '26', '5,483', '5,214', '858', '316', '2,008', '6,317', '372', '129', '161', '529', '60', '261', '3,130', '451', '248', '999', '6', '521', '16

quality data는 ,가 포함된 숫자이므로 숫자형으로 바꿔주어야한다.

In [ ]:
quality_df[quality_df['time'].str.contains('20201130.{6}')].drop(['fwver','user_id'],axis=1).astype(float).describe()

In [ ]:
quality_ppdf = quality_df[quality_df['time'].str.contains('20201130.{6}')].drop_duplicates()